In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, ifft
from scipy.io import wavfile as wav
import wave

π = np.pi

# **I] <u>Les séries de Fourier</u>**

## **1) <u>Les paramètres d'un signal périodique : fréquence (Hz), amplitude et position de phase (rad)</u>**

#### **<u>Note</u>** : <br> La fréquence $f$ correspond à l'inverse de la période $T$ (s). On a donc $f = 1/T$. <br> On peut également définir la pulsation $ω$ (rad/s) telle que $ω = 2πf = 2π/T$.

In [ ]:
t = np.arange(0, 2, 0.00001)  # Etalon pour l'axe des abscices
f1 = np.sin(2*π*t)            # Sinusoïde étalon d'amplitude 1, de fréquence 1/t et de position de phase 0°
f2 = np.sin(2*π*t*2)          # Sinusoïde de fréquence 2/t
f3 = 2*f1                     # Sinusoïde d'amplitude 2
f4 = np.sin(2*π*t+π/2)        # Sinusoïde de position de phase π/2

In [ ]:
# Démonstration de ce qu'est la fréquence (ou de la période)

plt.figure(figsize=(12, 5))
plt.plot(t, f1, label='f1')
plt.plot(t, f2, label='f2')
plt.axhline(y=0, color='black', linestyle='-')
plt.axvline(x=0, color='black', linestyle='-')
plt.xlabel('t (s)', fontsize=15)
plt.ylabel('f(t)', rotation=0, fontsize=15)
plt.title('\nf1 a une fréquence de 1Hz\nf2 a une fréquence de 2Hz\n', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
# Démonstration de ce qu'est l'amplitude

plt.figure(figsize=(12, 5))
plt.plot(t, f1, label='f1')
plt.plot(t, f3, label='f3')
plt.axhline(y=0, color='black', linestyle='-')
plt.axvline(x=0, color='black', linestyle='-')
plt.xlabel('t (s)', fontsize=15)
plt.ylabel('f(t)', rotation=0, fontsize=15)
plt.title('\nf1 a une amplitude de 1\nf3 a une amplitude de 2\n', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
# Démonstration de ce qu'est la position de phase

plt.figure(figsize=(12, 5))
plt.plot(t, f1, label='f1')
plt.plot(t, f4, label='f4')
plt.axhline(y=0, color='black', linestyle='-')
plt.axvline(x=0, color='black', linestyle='-')
plt.xlabel('t (s)', fontsize=15)
plt.ylabel('f(t)', rotation=0, fontsize=15)
plt.title('\nf1 a une position de phase de 0°\nf4 a une position de phase de 90° (π/2)\n', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

## **2) <u>Qu'est-ce qu'une série de Fourier ?</u>**

### Un signal périodique de fréquence $f$ et de forme quelconque peut être obtenu en ajoutant à une sinusoïde de fréquence $f$ (appelée **fondamentale**), des sinusoïdes dont les fréquences sont des multiples entiers de $f$ (appelées **harmoniques**). Ces signaux ont des amplitudes et des positions de phase appropriées. Les éléments de cette somme constituent la série de Fourier du signal.
### A chaque fréquence présente dans la série de Fourier est associé un coefficient. <br> On appelle l'ensemble de ces coefficients les **coefficients de Fourier** de la fonction.

## **<u>3) Création d'un signal carré à partir de sa série de Fourier</u>**

In [ ]:
# Démonstration de ce qu'est un signal carré

plt.figure(figsize=(12, 6))
plt.plot(t, signal.square(2*π*t), linewidth=2)
plt.axhline(y=0, color='black', linewidth=0.5)
plt.axvline(x=0, color='black', linewidth=0.5)
plt.xticks([])
plt.yticks([-1, 0, 1])
plt.show()

In [ ]:
# Approximation d'un signal carré de fréquence 1 Hz avec les premiers éléments de sa série de Fourier

y0 = (4/π)*np.sin(2*π*t)      # Premier élément (fréquence fondamentale : f = 1 Hz ; amplitude : A = 4/π)
y1 = (4/π)*np.sin(6*π*t)/3    # Deuxième élément (première harmonique : 3f ; amplitude : A/3)
y2 = (4/π)*np.sin(10*π*t)/5   # Troisième élément (deuxième harmonique : 5f ; amplitude : A/5)
y3 = (4/π)*np.sin(14*π*t)/7   # Quatrième élément (troisième harmonique : 7f ; amplitude : A/7)

In [ ]:
# Tracé des 4 premiers éléments séparément

plt.figure(figsize=(20, 8), dpi=100)
c = 1
for y in [y0, y1, y2, y3]:
    plt.subplot(1, 4, c)
    plt.plot(t[:100001], y[:100001], label=f'y{c-1}', color=f'C{c-1}')
    plt.legend(fontsize=15)
    plt.xlabel('t (s)', fontsize=15)
    plt.yticks(np.arange(-1.5, 2, .5))
    plt.grid()
    plt.axhline(y=0, color='black', linestyle='-')
    plt.axvline(x=0, color='black', linestyle='-')
    c += 1
plt.suptitle('\n4 premiers éléments de la série de Fourier du signal carré de fréquence 1 Hz', fontsize=20)
plt.show()

In [ ]:
def decomp(lst):
    y = sum(i for i in lst)
    plt.figure(figsize=(12, 5))
    plt.plot(t, y)
    plt.plot(t, signal.square(2*π*t))
    plt.xlabel('t (s)', fontsize=15)
    plt.ylabel('f(t)', rotation=0, fontsize=15)
    plt.title(f'\nSomme des {len(lst)} premiers éléments de la série de Fourier pour un signal carré\n', fontsize=20)
    plt.grid()
    plt.show()

In [ ]:
decomp([y0])

### Avec : $y_0(t) = \frac{4}{π}sin(2πft)$

In [ ]:
decomp([y0, y1])

### Avec : $y_1(t) = \frac{4}{π}×\frac{1}{3}sin(6πft)$

In [ ]:
decomp([y0, y1, y2])

### Avec : $y_2(t) = \frac{4}{π}×\frac{1}{5}sin(10πft)$

In [ ]:
decomp([y0, y1, y2, y3])

### Avec : $y_3(t) = \frac{4}{π}×\frac{1}{7}sin(14πft)$

### Formule de la série de Fourier du signal carré (avec $f$ la fréquence du signal) :
# $$ x_{carré}(t) = \frac{4}{π}\sum_{k=0}^\infty\frac{sin((2k+1)2πft)}{2k+1} $$ $$ = \frac{4}{π}(sin(2πft)+\frac{1}{3}sin(6πft)+\frac{1}{5}sin(10πft)+...) $$

In [ ]:
# Généralisation

def signal_carré(n, ax, f=1):
    y = 0
    for k in range(n+1): y += 4*np.sin((2*k+1)*2*π*f*t)/(2*k+1)/π
    ax.plot(t, y)
    ax.set(xlabel='t (s)', ylabel='f(t)')
    ax.title.set_text(f'Somme des {n} premiers éléments de la série de Fourier pour un signal carré')
    ax.grid()

def plotter(N):
    fig, axes = plt.subplots(2, round(len(N)/2), figsize=(20, 10), dpi=300)
    ax_y = 0 ; ax_x = 0
    for n in N:
        signal_carré(n, ax=axes[int(ax_y), ax_x])
        ax_x = 0 if ax_x else 1
        ax_y += 0.5
    fig.show()

In [ ]:
N = [5, 10, 25, 250]
plotter(N)

### **<u>Note</u> :** Dans le cas d'un signal carré, on observe une déformation du signal appelée "phénomène de Gibbs". C'est un effet de bord qui se produit à proximité d'une discontinuité (passage d'un plateau à un autre). La méthode de fenêtrage de l'approximation sigma permet de réduire cet effet en atténuant les coefficients de Fourier d'ordre élevé qui rendent la série localement divergente.

## **4) <u>Spectre d'amplitude d'un signal T-périodique</u> :** l'exemple du signal carré de fréquence $f = 1 Hz$

### On peut représenter le développement en série de Fourier d'un signal T-périodique en traçant le **spectre** d'amplitude de ce signal. Pour ce faire, on représente l'amplitude $A_n$ de chaque élement de la série de Fourier du signal en fonction de sa fréquence $f_n$ (ou de sa pulsation $ω_n$). <br><br> Voici ce que l'on obtient dans le cas du signal carré vu précedemment :

In [ ]:
# Tracé du spectre d'amplitude du signal carré

def spectre_signal_carré(n, f=1):
    frequence = []
    amplitude = []
    for k in range(n+1):
        frequence.append((2*k+1)*f)
        amplitude.append(4/(2*k+1)/π)
    plt.figure(figsize=(20, 10))
    plt.bar(x=frequence, height=amplitude, width=0.1)
    plt.xticks(np.arange(min(frequence), max(frequence)+1, 1))
    plt.yticks(np.arange(round(min(amplitude), 1), round(max(amplitude), 1), 0.1))
    plt.xlabel('fn (Hz)', fontsize=15)
    plt.ylabel('An     ', rotation=0, fontsize=15)
    plt.title(f"\nSpectre d'amplitude d'un signal carré de fréquence {f} Hz\n", fontsize=20)
    plt.show()

In [ ]:
spectre_signal_carré(10)

### On remarque que ce spectre est discontinu, constitué de raies. On dira qu'il est **discret**.
### Dans ce graphique : 
### &emsp; - la barre $f_0 = 1 Hz$ correspond à $y_0$, <br> &emsp; - la barre $f_1 = 3 Hz$ à $y_1$, <br> &emsp; - la barre $f_2 = 5 Hz$ à $y_2$, <br> &emsp; - la barre $f_3 = 7 Hz$ à $y_3$. <br>

<center> <img src="Ressources/Fourier_transform_time_and_frequency_domains_%28small%29.gif"/> </center>

## **5) <u>Généralisation du développement en série de Fourier à toutes les fonctions T-périodiques</u>**

### On peut généraliser ce principe à toute fonction $f$ T-périodique définie dans $ℝ$ grâce à la formule suivante :
# $$f(x) = \sum_{n=-\infty}^{+\infty}c_n(f)e^{i2π\frac{n}{T}x}$$

### avec $c_{n}(f)$ les coefficients de Fourier de $f$ définis par :
# $$c_n(f) = \frac{1}{T}\int_{T}f(t)e^{-i2π\frac{n}{T}t}dt $$

### On utilise ici la forme exponentielle, qui peut être développée selon la formule d'Euler en :
# $$ e^{ix} = cos(x)+isin(x) $$
### avec $i$ l'unité imaginaire telle que $i^2 = -1$

### Les séries de Fourier ont de multiples applications mathématiques, notamment dans le calcul de séries numériques, la résolution de certaines équations différentielles linéraires ou encore en permettant de tirer des inégalités intéressantes entre certains types de fonctions et leur dérivée. <br><br>

-----------------------------------------------------------------------------------------------------------------------------------------------------------

# **II] <u>La Transformation de Fourier</u>**

## **1) <u>Introduction</u>**

### On a vu que l'on pouvait développer en série de Fourier un signal périodique, mais qu'en est-il des signaux non périodiques ? Il suffit de considérer qu'un signal non périodique est périodique mais avec une période de durée infinie !<br><br> La transformation de Fourier est une extension, pour les fonctions non périodiques, du développement en série de Fourier des fonctions périodiques. Elle associe à une fonction intégrable définie sur $ℝ^n$ et à valeurs réelles ou complexes, une autre fonction sur $ℝ^n$ appelée transformée de Fourier dont la variable indépendante peut s'interpréter en physique comme la fréquence ou la pulsation. <br><br> La transformée de Fourier représente une fonction de la densité spectrale de la fonction dont elle provient, en tant que moyenne de fonctions trigonométriques de toutes fréquences. <br>
### On appelle spectre fréquentiel d'un signal la courbe de sa transformée de Fourier.

### Ainsi, si $f$ est une fonction définie et intégrable sur $ℝ$ sa transformée de Fourier $\hat{f}$ est donnée par la formule :
# $$ \hat{f}(ξ)=\int_{-\infty}^{+\infty}f(x)e^{-iξx}dx $$
### avec $ξ$ la fréquence

### Si $\hat{f}$ est elle-même intégrable sur $ℝ$, la formule de transformation de fourier inverse permet de retrouver $f$ selon la formule :
# $$ f(x)=\int_{-\infty}^{+\infty}\hat{f}(ξ)e^{2πixξ}dξ $$

### Ces formules ont différentes généralisation pour différents ensembles de définition. <br> On peut retenir que pour $μ$ une fonction définie et intégrable sur $ℝ^n$ on a :
# $$ \hat{μ}(ξ) = \int_{ℝ^n}e^{-2πixξ}dμ $$

### En particulier sur $ℝ^2$, on utilisera notamment pour le traitement d'images :
# $$ \hat{f}(u,v) = \int_{-\infty}^{+\infty}\int_{-\infty}^{+\infty}f(x,y)e^{-i(ux+vy)}dxdy $$

## **2) <u>Les applications de la transformation de Fourier</u>**

### Les principales applications pratiques de la transformation de Fourier sont :
### • La résolution d'équations différentielles partielles ;
### • La spectroscopie (mesure d'un spectre d'émission ou d'absorbtion) : <br> &emsp; - spectroscopie optique, <br> &emsp; - spectroscopie IR (infra-rouge), <br> &emsp; - spectroscopie RMN (résonance magnétique nucléaire), <br> &emsp; - spectrométrie de masse, <br> &emsp; - spectroscopie de résonance de spin électronique. <br>
### • La physique quantique ;
### • Le **traitement de signal**, notamment l'analyse spectrale de séries temporelles (en pratique on n'appliquera pas une transformation de Fourier directement sur le signal mais sur sa fonction d'autocorrélation).

## **3) <u>La transformée de Fourier discrète (DFT)</u>**

### La transformation de Fourier discrète permet de convertir une séquence finie d'échantillons équidistants d'une fonction en une séquence de même taille d'échantillons équidistants de sa transformée de Fourier discrète.
### La transformée de Fourier discrète est une fonction à valeurs complexe (de la forme $a + bi$ avec $(a, b) ∈ ℝ^2$ et $i$ tel que $i^2 = -1$) de la fréquence.
### L'intérêt d'une transformation de Fourier discrète est qu'elle traite une quantité finie de données, et peut donc être implémentée dans des algorithmes numériques.
### En traitement numérique du signal, cette fonction représente la variation de toute quantité par rapport au temps, telle que l'amplitude d'une onde sonore (pression), d'un signal radio (champ électromagnétique), de températures... <br> Dans le traitement d'images, les échantillons peuvent être les valeurs des pixels selon une ligne ou une colonne.

### La formule de la DFT multidimentionnelle :
# $$ X_k = \sum_{n=0}^{N-1}e^{-2πik(n/N)}x_n $$

-----------------------------------------------------------------------------------------------------------------------------------------------------------

# **III] <u>La transformation de Fourier rapide (FFT)</u>**

## **1) <u>Introduction</u>**

### La FFT est un algorithme de calcul de la DFT ou son inverse (IDFT) d'une séquence. Sa complexité varie en $O(n×log(n))$ tandis que celle de l'algorithme de DFT "naïf" varie en $O(n^2)$.

## **2) <u>Application algorithmique sur un enregistrement</u>**

### Nous avons un fichier audio et nous voulons enlever le bruit et le compresser. <br><br> Voici le prodédé que nous allons utiliser : $\begin{bmatrix} | \\ f \\ | \end{bmatrix}$ - FFT&rarr; $\begin{bmatrix} | \\ \hat{f} \\ | \end{bmatrix}$ -filtre&rarr; $\begin{bmatrix} | \\ \hat{f}_{filt} \\ | \end{bmatrix}$ -IFFT&rarr; $\begin{bmatrix} | \\ {f}_{filt} \\ | \end{bmatrix}$

In [ ]:
# On récupère notre signal

fs, data = wav.read('Ressources/original.wav')  # Avec fs la fréquence d'échantillonnage
c1 = data.T[0]                                  # L'enregistrement a deux channels, on récupère le premier

plt.plot(c1)
plt.show()

In [ ]:
# Calculons la transformée de Fourier de ce signal

n = len(c1)
fhat = fft(c1)                                # On calcule la FFT de c1
PSD = fhat*np.conj(fhat)/n                    # On calcule la densité spectrale de puissance
freq = (1/(fs*n))*np.arange(n)                # On construit le vecteur fréquentiel
L = np.arange(1, np.floor(n/2), dtype='int')  # Le signal est symétrique, on peut ne tracer que la moitié

In [ ]:
# Traçons la transformée de Fourier de ce signal

l = len(fhat)
k = np.arange(l)          # On créé la liste des entiers naturels de la taille du nombre de points à tracer
T = len(data)/fs          # T est notre période : le nombre d'échantillons divisé par la fréquence d'échantillonnage
freqs = k/T               # Liste des fréquences
x = freqs[:int(l/2)]      # On construit l'axe des absisses à partir de la fréquence des échantillons
y = abs(fhat[:int(l/2)])  # On normalise et on ne garde que la première moitié des points (fhat est symétrique autour de x=24000Hz)

plt.figure(figsize=(12, 5), dpi=100)
plt.plot(x, y)
plt.xlabel('f (Hz)', fontsize=15)
plt.ylabel('Amplitude', fontsize=15)
plt.title('\nTransformée de Fourier\n', fontsize=20)
plt.show()

In [ ]:
# Comme on peut le voir, le signal est quasi nul passé 5000 Hz. Zoomons sur la partie qui nous intéresse

plt.figure(figsize=(12, 5), dpi=100)
plt.plot(x[:50000], y[:50000])
plt.xlabel('f (Hz)', fontsize=15)
plt.ylabel('Amplitude', fontsize=15)
plt.title('\nTransformée de Fourier entre 0Hz et 2600Hz\n', fontsize=20)
plt.show()

In [ ]:
# Zoomons sur le pic le plus élevé du spectre

plt.figure(figsize=(12, 5), dpi=100)
plt.plot(x[1750:2100], y[1750:2100])
plt.xlabel('f (Hz)', fontsize=15)
plt.ylabel('Amplitude', fontsize=15)
plt.title('\nTransformée de Fourier entre 90 et 110 Hz\n', fontsize=20)
plt.show()

In [ ]:
# Filtrons le signal en ne gardant que les fréquences dont l'amplitude est supérieure à un certain seuil

indices = PSD > 15                                              # On créé une liste de booléens répertoriant par indice si l'amplitude de la fréquence est au dessus ou en dessous d'un certain seuil
fhatclean = [fhat[i] for i in range(len(fhat)) if indices[i]]   # On enlève ces valeurs de fhat
c1filt = ifft(fhatclean)                                        # On applique la transformée de Fourier rapide inverse sur ce nouveau signal

In [ ]:
# Comparons le signal original et le signal après filtrage

c1f = ifft(fhat*indices).real  # On créé ici c1f en lieu de c1filt en remplaçant les données supprimées par des 0 afin de pouvoir comparer le signal avant et après filtrage

plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
plt.plot(c1)
plt.title('c1', fontsize=15)
plt.subplot(1, 3, 2)
plt.plot(c1f, color='C1')
plt.title('c1f', fontsize=15)
plt.subplot(1, 3, 3)
plt.plot(c1-c1f, color='C2')
plt.title('c1 - c1f', fontsize=15)
plt.show()

In [ ]:
# Vérifions que la répartition en amplitude de c1 - c1f vérifie bien une loi normale

plt.figure(figsize=(10, 5))
plt.hist(c1-c1f, bins=10000)
plt.xlim(-3, 3)
plt.title('Répartition en amplitude de c1 - c1f', fontsize=20)
plt.xlabel('Amplitude', fontsize=15)
plt.show()

In [ ]:
# Exportons le signal obtenu

wav.write('nouveau.wav', fs, c1filt.real.astype(np.int16))

In [ ]:
# Les fréquences produites par l'IFFT dépendent du nombre d'échantillons, le signal résultant est donc compressé en fréquence. On doit l'étirer pour retrouver le signal initial

factor = n/len(c1filt)
infile=wave.open('nouveau.wav', 'rb')
rate= infile.getframerate()
channels=infile.getnchannels()
swidth=infile.getsampwidth()
nframes= infile.getnframes()
audio_signal= infile.readframes(nframes)
outfile = wave.open('stretched.wav', 'wb')
outfile.setnchannels(channels)
outfile.setsampwidth(swidth)
outfile.setframerate(rate/factor)
outfile.writeframes(audio_signal)
outfile.close()

## **3) <u>Importance de l'algorithme de FFT par rapport à la DFT naïve</u> :** Comparaison de $n$, $n×log_2(n)$ et $n^2$

### <u> Rappel</u> : La fonction $f(x) = log_2(x)$ est la réciproque de $g(x) = 2^x$.
### Ainsi : <br> &emsp;&emsp;$g(0) = 2^0 = 1$ &emsp;$=>$&emsp; $f(1) = 0$ <br> &emsp;&emsp;$g(1) = 2^1 = 2$ &emsp;$=>$&emsp; $f(2) = 1$ <br> &emsp;&emsp;$g(2) = 2^2 = 4$ &emsp;$=>$&emsp; $f(4) = 2$ <br> &emsp;&emsp;$...$

In [ ]:
def compare(n):
    sqr = [i*i for i in np.arange(1, n+1)]
    nlogn = [i*np.log2(i) for i in np.arange(1, n+1)]
    N = [i for i in np.arange(1, n+1)]
    plt.figure(figsize=(10, 5))
    plt.plot(sqr, label='f(n) = n^2')
    plt.plot(nlogn, label='f(n) = n*log(n)')
    plt.plot(N, label='f(n) = n')
    plt.xlabel('n', fontsize=15)
    plt.ylabel('f(n)', rotation=0, fontsize=15)
    plt.ticklabel_format(style='plain')
    plt.title(f'Comparaison pour n = {n}', fontsize=20)
    plt.legend(loc='upper left', bbox_to_anchor=(0.15, -0.1, 0.5, 0.9), fontsize=15)
    plt.show()
    d = {'Résultat' : [n, round(nlogn[-1]), sqr[-1]]}
    return pd.DataFrame(data=d, index=['n', 'n*log(n)', 'n^2'])

def compare2(n):
    h, m = divmod(round(n/np.log(n)), 60)
    d, h = divmod(h, 24)
    y, d = divmod(d, 365)
    time = f'{m} minutes'
    if h: time = f'{h} heures ' + time
    if d: time = f'{d} jours ' + time
    if y: time = f'{y} années ' + time
    print(f'Si algorithme en O(n*log(n)) mettait 1 min pour effectuer {n} oppérations, un algorithme équivalent en O(n^2) mettrait {time}.')

In [ ]:
compare(10)
compare2(10)

In [ ]:
compare(100)
compare2(100)

In [ ]:
compare(1000)
compare2(1000)

In [ ]:
compare2(1000000) # n = 1 million

In [ ]:
compare2(1000000000) # n = 1 milliard

## Remerciements : Wikipédia, Stack Overflow, Jonathan
#### • Les séries de Fourier : https://en.wikipedia.org/wiki/Fourier_series <br><br> • La transformation de Fourier : https://en.wikipedia.org/wiki/Fourier_transform <br><br> • La transformation de Fourier discrète : https://en.wikipedia.org/wiki/Discrete_Fourier_transform <br><br> • La tranformation de Fourier rapide : https://en.wikipedia.org/wiki/Fast_Fourier_transform <br><br> • Vidéo sur l'importance de la FFT : https://www.youtube.com/watch?v=nmgFG7PUHfo <br><br> • Vidéo sur le débruitage de données avec la FFT en python https://www.youtube.com/watch?v=s2K1JfNR7Sc